In [1]:
from stanza.runtime import setup
setup()

from stanza.data.sequence import SequenceData, Chunk
from stanza.env.mujoco.reacher import ReacherPosObs, ReacherEnv, PositionalControlTransform, PositionalObsTransform
from stanza.env import ImageRender
from stanza import canvas
from stanza.policy.transforms import Transform, ChainedTransform

import jax
import jax.numpy as jnp
from jax.sharding import Mesh, NamedSharding, PartitionSpec

from functools import partial

In [2]:
if len(jax.devices()) > 1:
    render_mesh = Mesh(jax.devices()[:8], ('x',))
    render_sharding = NamedSharding(render_mesh, PartitionSpec('x'))
    data_mesh = Mesh(jax.devices()[:8], ('x',))
    data_sharding = NamedSharding(data_mesh, PartitionSpec('x'))

env = ReacherEnv()
env = ChainedTransform([
    PositionalControlTransform(),
    PositionalObsTransform()
]).transform_env(env)

def draw(action_chunk):
    T = action_chunk.shape[0]
    colors = jnp.array((jnp.arange(T)/T, jnp.zeros(T), jnp.zeros(T))).T
    circles = canvas.fill(
        canvas.circle(action_chunk, 0.009*jnp.ones(T)),
        color=colors
    )
    circles = canvas.stack_batch(circles)
    circles = canvas.transform(circles,
        translation=(0.25,-0.25),
        scale=(512, -512)
    )
    return circles

@partial(jax.jit, out_shardings=render_sharding)
def render(state, action_chunks):
    image = env.render(ImageRender(256, 256), state)
    if action_chunks is not None:
        circles = canvas.stack_batch(jax.vmap(draw)(action_chunks))
        return canvas.paint(image, circles)
    return image

In [3]:
data = SequenceData.load("reacher_data.pkl")

obs_length = 1
action_length = 6
def map_chunks(chunk : Chunk):
    state, action = chunk.chunk
    state = jax.vmap(env.observe)(state)
    obs = jax.tree_util.tree_map(lambda x: x[:obs_length], state)
    # use the future state positions as the actions
    action = jax.tree_util.tree_map(lambda x: x[-action_length:], state.fingertip_pos)
    # action = jax.tree_util.tree_map(lambda x: x[-action_length:], action)
    return obs, action
data = data.chunk(obs_length + action_length).map(map_chunks).as_pytree()
data_len = (data[1].shape[0]//8)*8    # truncate length of data to multiple of 8 for sharding
data = jax.tree_util.tree_map(lambda x: x[:data_len], data)
data = jax.device_put(data, data_sharding)
#jax.debug.visualize_array_sharding(data[1][:,:,0])
#jax.debug.visualize_array_sharding(data[1][:,:,1])
#jax.debug.visualize_array_sharding(data[1][210])

sample_chunk = jax.tree_util.tree_map(lambda x: x[0], data)

print("Loaded data", jax.tree_util.tree_map(lambda x: x.shape, data))

Loaded data (ReacherPosObs(fingertip_pos=(760, 1, 2), body0_pos=(760, 1, 2), body0_rot=(760, 1), body1_pos=(760, 1, 2), body1_rot=(760, 1)), (760, 6, 2))


In [4]:
from stanza.diffusion import DDPMSchedule, nonparametric
from stanza.policy import PolicyInput, PolicyOutput
from stanza.policy.transforms import ChunkTransform
from jax.random import PRNGKey

schedule = DDPMSchedule.make_squaredcos_cap_v2(16, prediction_type="sample")

@jax.jit
def chunk_policy(input: PolicyInput) -> PolicyOutput:
    obs = input.observation
    #estimator = nonparametric.nw_local_poly_closed(data, schedule, 1, 0.04)
    estimator = nonparametric.nw_local_poly(PRNGKey(42), data, schedule, 1, 
                              nonparametric.log_gaussian_kernel, nonparametric.log_gaussian_kernel, 
                              0.1, 0.1, 64)
    diffuser = nonparametric.nw_diffuser(obs, estimator)
    #diffuser = nonparametric.nw_cond_diffuser(obs, data, schedule, nonparametric.log_gaussian_kernel, 0.5)
    #diffuser = nonparametric.closest_diffuser(obs, data)
    action = schedule.sample(input.rng_key, diffuser, sample_chunk[1])
    return PolicyOutput(action)
transform = ChunkTransform(obs_length, action_length)
policy = transform.transform_policy(chunk_policy)

In [5]:
from stanza.util.ipython import as_image

state = env.reset(jax.random.key(0))
state_batch = jax.tree_util.tree_map(
    lambda x: jnp.repeat(x[None], obs_length, 0), 
    state
)
output = chunk_policy(PolicyInput(
    jax.vmap(env.observe)(state_batch),
    rng_key=jax.random.key(42))
).action 
print(state, output)

as_image(render(state, output[None,...]))

ReacherState(q=Array([ 0.35943913, -2.4795399 ,  0.1       , -0.1       ], dtype=float32), qd=Array([0., 0., 0., 0.], dtype=float32)) [[ 2.52637733e-02 -7.85992295e-02]
 [ 1.64892953e-02 -8.37052166e-02]
 [ 9.52803995e-03 -8.84022638e-02]
 [ 3.90637014e-03 -9.26109776e-02]
 [ 9.42464831e-05 -9.66746286e-02]
 [-1.71170046e-03 -1.00243635e-01]]
(256, 256, 4)


In [6]:
#%timeit chunk_policy(PolicyInput(jax.vmap(env.observe)(state_batch), rng_key=jax.random.key(42))).action 

In [7]:
from stanza.policy.ipython import StreamingInterface
from threading import Thread
from ipywidgets import Label
import time


@partial(jax.jit, out_shardings=render_sharding)
def batch_policy(obs, rng_key):
    keys = jax.random.split(rng_key, 8)
    return jax.vmap(chunk_policy, in_axes=(PolicyInput(None, rng_key=0),))(
        PolicyInput(obs, rng_key=keys)
    ).action

label = Label(value="Hello world")
interactive = StreamingInterface(256, 256)
def loop():
    state = env.reset(jax.random.key(42))
    state_batch = jax.tree_util.tree_map(
        lambda x: jnp.repeat(x[None], obs_length, 0), 
        state
    )
    key = jax.random.key(42)
    while True:
        t = time.time()
        key, r = jax.random.split(key)
        action = interactive.mouse_pos()
        prev_state = state
        state = env.step(state, action)
        reward = env.reward(prev_state, action, state)
        label.value = f"reward: {reward}"
        state_batch = jax.tree_util.tree_map(
            lambda x, s: jnp.roll(x, -1).at[-1].set(s),
            state_batch, state
        )
        obs = jax.vmap(env.observe)(state_batch)
        output = batch_policy(obs, jax.random.key(42))
        output = jax.tree_util.tree_map(lambda x: x[:1], output)
        image = render(state, output)
        interactive.update(image)
        elapsed = time.time() - t
        time.sleep(max(0, 1/30 - elapsed))
t = Thread(target=loop, daemon=True)
t.start()
t.__del__ = lambda: t.stop()
display(label)
interactive

Label(value='Hello world')